In [74]:
import pygame, random, math
import numpy as np
import csv, os
import matplotlib.pyplot as plt
import pandas as pd

pygame.init()
font_size = 14
font = pygame.font.SysFont('Arial', font_size)

In [75]:
WIDTH, HEIGHT = 800, 800
FPS = 60
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (30, 144, 255)
SIZE_R = 10
SPEED = 1
PREY_COUNT = 25
PREY_MAX_SIZE = 300
PREY_MIN_SIZE = 50
PRED1_COUNT = 2
PRED2_COUNT = 2
MOVEMENT = 5
STARTING_ENERGY = 1000
BASE_METABOLISM = 5
PROB_SMALLEST = 0.8
PROB_LARGEST = 0.2

In [76]:
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

In [77]:
sizes = np.arange(PREY_MIN_SIZE, PREY_MAX_SIZE + 1, 1)
m = (PROB_LARGEST - PROB_SMALLEST) / (PREY_MAX_SIZE - PREY_MIN_SIZE)
m = round(m, 5)
a = PROB_SMALLEST - m * PREY_MIN_SIZE
probs = [round(m * size + a, 5) for size in sizes]
total_probs = sum(probs)
normalized_probs = [prob / total_probs for prob in probs]
colors = np.arange(255, 80, (80 - 255) / len(sizes))
color_size = dict(zip(sizes, colors))
class Prey:
    def __init__(self, position, prey_list):
        x,y = position
        self.x = x
        self.y = y
        self.size = random.choices(sizes, weights=normalized_probs)[0]
        while True:
            if self.size in [prey.size for prey in prey_list]:
                self.size = random.choices(sizes, weights=normalized_probs)[0]
            else:
                break
        self.energy = self.size * 20
        self.handling = self.size 
        self.r = SIZE_R * self.size / PREY_MAX_SIZE
        self.color = (70, color_size[self.size], 70)
    
    def gen(prey_number):
        prey_list = []
        while len(prey_list) < prey_number:
            point = (random.randrange(0, WIDTH), random.randrange(0, HEIGHT))
            if point not in prey_list:
                prey_list.append(Prey(point, prey_list))
        return prey_list
    
    def draw(prey_list, screen):
        for prey in prey_list:
            pygame.draw.circle(screen, prey.color, (prey.x, prey.y), prey.r)

In [78]:
class Predator:
    def __init__(self, position, energy, pred_type):
        x,y = position
        self.x = x
        self.y = y
        self.energy = energy
        self.type = pred_type
        self.is_eating = None
        self.eating_prey = None
        self.eat_time_left = 0

    def gen(pred1, pred2):
        pred_list = []
        while len(pred_list) < pred1:
            point = (random.randrange(0, WIDTH), random.randrange(0, HEIGHT))
            if point not in pred_list:
                pred_list.append(Predator(point, STARTING_ENERGY, "OPT"))
        while len(pred_list)  < pred1 + pred2:
            point = (random.randrange(0, WIDTH), random.randrange(0, HEIGHT))
            if point not in pred_list:
                pred_list.append(Predator(point, STARTING_ENERGY, "SPL"))    
        return pred_list
    
    def move(self, target, preys):
        if self.is_eating:
            return
        pygame.draw.line(screen, WHITE, (self.x, self.y,) , target)
        tx , ty = target
        direction_x = tx - self.x
        direction_y = ty - self.y
        length = math.sqrt(direction_x ** 2 + direction_y ** 2)
        if length != 0:
            direction_x /= length
            direction_y /= length
        self.x += SPEED * direction_x
        self.y += SPEED * direction_y
        self.energy -= MOVEMENT * SPEED
        for prey in preys:
            if math.hypot(prey.x - self.x, prey.y - self.y) <= prey.r:
                if len(preys) == 1:
                    self.energy += prey.energy
                    preys.remove(prey)
                    break
                preys.remove(prey)
                self.is_eating = True
                self.eating_prey = prey
                self.eat_time_left = prey.handling
                break

    def eating(predators):
        for pred in predators:
            if pred.eat_time_left > 0:
                pred.energy += pred.eating_prey.energy / pred.eating_prey.handling
                pred.eat_time_left -= 1
            if pred.eat_time_left <= 0:
                pred.is_eating = False
                pred.eating_prey = None
    
    def optimal_foraging(predators, preys):
        for pred in predators:
            if not preys:
                continue
            if pred.type != "OPT":
                continue
            max_val = 0
            target_pos = (0,0)
            for prey in preys:
                distance  = math.sqrt((prey.x - pred.x) ** 2 + (prey.y - pred.y) ** 2)
                val = prey.energy / ((prey.handling * BASE_METABOLISM) + (distance * (MOVEMENT + BASE_METABOLISM)))
                if val > max_val:
                    max_val = val
                    target_pos = (prey.x, prey.y)
            if max_val > 0:
                pred.move(target_pos, preys)

    def simple_foraging(predators, preys):
        for pred in predators:
            if not preys:
                continue
            if pred.type != "SPL":
                continue
            min_dist = math.inf
            target_pos = (0,0)
            for prey in preys:
                distance  = math.sqrt((prey.x - pred.x) ** 2 + (prey.y - pred.y) ** 2)
                if min_dist > distance:
                    min_dist = distance
                    target_pos = (prey.x, prey.y)
            if min_dist > 0:
                pred.move(target_pos, preys)

    def draw(predators, screen):
        for pred in predators:
            if pred.type == "OPT":
                pygame.draw.circle(screen, RED, (pred.x, pred.y), SIZE_R)
                energy_text = font.render(str(int(pred.energy)) , True , WHITE)
                text_x = pred.x - energy_text.get_width() // 2
                text_y = pred.y - energy_text.get_width() // 2 - 15
                screen.blit(energy_text, (text_x, text_y))
            else:
                pygame.draw.circle(screen, BLUE, (pred.x, pred.y), SIZE_R)
                energy_text = font.render(str(int(pred.energy)) , True , WHITE)
                text_x = pred.x - energy_text.get_width() // 2
                text_y = pred.y - energy_text.get_width() // 2 - 15    
                screen.blit(energy_text, (text_x, text_y))



In [79]:
def main(simulation_time = 10):
    csv_file_path = str(simulation_time) + "_" + str(PREY_COUNT) + "_" + str(PRED1_COUNT) + "_" + str(PRED2_COUNT) + ".csv"
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
    with open(csv_file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([
                         "time",
                         "sim_number", 
                         "pred1_energy",
                         "pred1_mean_energy", 
                         "pred2_energy", 
                         "pred2_mean_energy", 
                         "prey_count",
                         "mean_prey_size",
                         "pred1_eating",
                         "pred2_eating"])
        run = True
        predators = []
        prey = []
        pause = True
        time_factor = 1.0
        sim = 0
        t = 1
        while run:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        pause = not pause
                    elif event.key == pygame.K_KP_PLUS:
                        time_factor = min(time_factor * 2 , 1000)
                    elif event.key == pygame.K_KP_MINUS:
                        time_factor = max(0.5, time_factor // 2)   
            if not prey:
                pause = True
                t = 1
                predators = []
                prey = Prey.gen(PREY_COUNT)
                predators = Predator.gen(PRED1_COUNT, PRED2_COUNT)
                pause = not pause
                sim += 1                    
                if sim > simulation_time:
                    run = False
                    pause = True
            screen.fill(BLACK)
            pygame.display.set_caption(
                "sim number: " + str(sim) + " of " + str(simulation_time) 
                + " | " + "time factor: " + str(time_factor) 
                + " | " + "paused: " + str(pause) 
                + " | " + "OPT:" + str(len([pred for pred in predators if pred.type == "OPT"])) + " mean:" + str(np.mean([pred.energy for pred in predators if pred.type == "OPT"]))
                + " | " + "SPL:" + str(len([pred for pred in predators if pred.type == "SPL"])) + " mean:" + str(np.mean([pred.energy for pred in predators if pred.type == "SPL"]))
                + " | " + "Prey Number:" + str(len(prey)) 
            )
            Prey.draw(prey, screen)
            Predator.draw(predators, screen)

            if not pause:
                Predator.optimal_foraging(predators, prey)
                Predator.simple_foraging(predators, prey)
                Predator.eating(predators)
                for pred in predators: 
                    pred.energy -= BASE_METABOLISM
                pred1_eating = []
                pred2_eating = []
                for pred in predators:
                    if pred.is_eating:
                        if pred.type == "OPT":
                            pred1_eating.append(pred.eating_prey.size)
                        if pred.type == "SPL":
                            pred2_eating.append(pred.eating_prey.size)
                pred1_eating_str = ", ".join(map(str, pred1_eating)) if pred1_eating else np.nan
                pred2_eating_str = ", ".join(map(str, pred2_eating)) if pred2_eating else np.nan
                writer.writerow([t, 
                                 sim,
                                ", ".join(str(pred.energy) for pred in predators if pred.type == "OPT"),
                                np.mean([pred.energy for pred in predators if pred.type == "OPT"]),
                                ", ".join(str(pred.energy) for pred in predators if pred.type == "SPL"),
                                np.mean([pred.energy for pred in predators if pred.type == "SPL"]),
                                len(prey),
                                np.mean([prey.size for prey in prey]),
                                pred1_eating_str,
                                pred2_eating_str
                                ])
                t += 1
            pygame.display.flip()
            clock.tick(FPS * time_factor)

        pygame.quit()

if __name__ == "__main__":
    main(500)

